In [55]:
import keras as ks
import tensorflow as tf
import numpy as np
import cv2 as cv
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [56]:
def create_image_data(root):
    images = []
    for subdir, dirs, files in os.walk(root):
        for file in files:
            read_file = subdir + "/" + file
            img = cv.imread(read_file,cv.IMREAD_GRAYSCALE)
            images.append(img)
    return np.array(images)

## Data Preprossesing

This will generate the image data and normalize the data to be between 0 and 1.
Darker pixels will be closer to 0 and lighter pixels will be closer to 0. Furthermore, we are using grey-scale .

In [57]:
frown = create_image_data("data/60frowns/")/255
smile = create_image_data("data/60smiles/")/255

frowns = frown.reshape(frown.shape[0],60,60,1).astype('float32')
smiles = smile.reshape(smile.shape[0],60,60,1).astype('float32')

X = np.concatenate([frowns, smiles])

In [58]:
frowns_y = np.repeat(0, frown.shape[0])
smiles_y = np.repeat(1, smile.shape[0])

y = np.concatenate([frowns_y,smiles_y])

Splits the data intro training and testing sets.

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Using the ImageDataGenerator we will create additional images for the model to trail on since we only have 360 images.

In [148]:
img_gen = ImageDataGenerator()
img_gen.fit(X_train)

## CNN Model
This will train and build the CNN model. 

### Layers
    1. Convolutional layer: 
        - Input shape: (60, 60, 1)
        - RelU activation function a' = max(0, a) for each pixel
        - L2 regularization with a = 0.001, multiply each weight in training by 0.001
    2. Pooling layer:
        - Pooling size = (3,3)
    3. Dense layer (Fully Connected layer):
        - Output size of 1
        - Sigmoid activation function
### Compiling the model
    - We will use a binary_crossentropy loss function
    - RMsprop optimizer function
### Fitting the model
#### ImageDataGenerator
    - A key detail about our model is that we will be generating 30 new images each epoch during the training of the model.

In [163]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras import regularizers
from keras import optimizers

In [164]:
model = ks.Sequential()

model.add(Conv2D(filters=15,kernel_size=(5,5),activation='relu', input_shape=(60,60,1), kernel_regularizer=regularizers.l2(0.001)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(units=1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='RMSprop', metrics = ['accuracy'])
history_model = model.fit_generator(img_gen.flow(X_train,y_train, batch_size=30),steps_per_epoch=45, epochs=5);
history_evaluate = model.evaluate(X_test, y_test);
history_evaluate

Epoch 1/5
45/45 [==============================] - 4s 94ms/step - loss: 0.4123 - acc: 0.8554
Epoch 2/5
45/45 [==============================] - 2s 34ms/step - loss: 0.1336 - acc: 0.9737
Epoch 3/5
45/45 [==============================] - 2s 34ms/step - loss: 0.0502 - acc: 0.9900
Epoch 4/5
45/45 [==============================] - 2s 34ms/step - loss: 0.0222 - acc: 0.9985
Epoch 5/5
108/108 [==============================] - 1s 12ms/step


[0.07220148446935194, 0.9814814792739021]

In [144]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_88 (Conv2D)           (None, 56, 56, 15)        390       
_________________________________________________________________
max_pooling2d_83 (MaxPooling (None, 28, 28, 15)        0         
_________________________________________________________________
flatten_83 (Flatten)         (None, 11760)             0         
_________________________________________________________________
dropout_58 (Dropout)         (None, 11760)             0         
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 11761     
Total params: 12,151
Trainable params: 12,151
Non-trainable params: 0
_________________________________________________________________
